In [1]:
import networkx as nx
import dwave_networkx as dnx

from dqanneal.graph.embed import (get_problem_graph, 
                                  embed_problem_onto_hardware, 
                                  define_embedded_qubo_problem,
                                  unembed_samples,
                                  build_reverse_annealing_schedule_embedding)
from dqanneal.solvers.solvers import dimod_optimizer_cloud, save_annealing_results

In [2]:
import dimod
import minorminer
import dwave # same as: dwave-ocean-sdk  
import dwaveoceansdk


print(dimod.__version__)
print(minorminer.__version__)
print(dwaveoceansdk.__version__)


0.12.18
0.2.16
8.1.0


# Define the QUBO problem

In [3]:
dwave_qubo = {
    (0, 0):  -1,
    (0, 1):   2,
    (0, 2):  -3,
    (0, 3):   4,
    (0, 4):  -5,
    (0, 5):   6,
    (0, 6):  -7,
    (0, 7):   8,
    (0, 8):  -9,
    (0, 9):   10,
    (0, 10): -11,
    (0, 11):  12,
    (0, 12): -13,
    (0, 13):  14,
    (0, 14): -15,
    (0, 15):  16,
    (0, 16): -17,
    (0, 17):  18,
    (0, 18): -19,
    (0, 19):  20,
    (0, 20): -21,
    (4, 16):  22,
    (1, 19): -23,
    }

In [4]:
source_bqm = dimod.BinaryQuadraticModel.from_qubo(dwave_qubo)
print(f'numbera of variables: {len(source_bqm.variables)}')

numbera of variables: 21


In [5]:
problem_graph, problem_edge_dict = get_problem_graph(dwave_qubo)
nx.get_edge_attributes(problem_graph, "bias")

{(0, 1): np.float64(2.0),
 (0, 2): np.float64(-3.0),
 (0, 3): np.float64(4.0),
 (0, 4): np.float64(-5.0),
 (0, 5): np.float64(6.0),
 (0, 6): np.float64(-7.0),
 (0, 7): np.float64(8.0),
 (0, 8): np.float64(-9.0),
 (0, 9): np.float64(10.0),
 (0, 10): np.float64(-11.0),
 (0, 11): np.float64(12.0),
 (0, 12): np.float64(-13.0),
 (0, 13): np.float64(14.0),
 (0, 14): np.float64(-15.0),
 (0, 15): np.float64(16.0),
 (0, 16): np.float64(-17.0),
 (0, 17): np.float64(18.0),
 (0, 18): np.float64(-19.0),
 (0, 19): np.float64(20.0),
 (0, 20): np.float64(-21.0),
 (1, 19): np.float64(-23.0),
 (4, 16): np.float64(22.0)}

# build a hardware graph

In [ ]:
### need to setup dwave leap token access for this

# from dwave.system.samplers import DWaveSampler
# import networkx as nx
# sampler_dw = DWaveSampler(solver={'topology__type': 'pegasus'})
# G = nx.from_dict_of_lists(sampler_dw.adjacency)
# edge_dict = {node: set(G.neighbors(node)) for node in G.nodes}

In [6]:
# https://docs.ocean.dwavesys.com/en/latest/docs_dnx/reference/generators.html
# G = dnx.chimera_graph(1, 1, 4)
# G = dnx.zephyr_graph(6)

# https://github.com/dwave-examples/pegasus-notebook/blob/master/01-exploring-pegasus.ipynb
hardware_graph = dnx.pegasus_graph(16)
hardware_graph_edge_dict = {node: set(hardware_graph.neighbors(node)) for node in hardware_graph.nodes}

# embed given problem onto hardware graph

In [7]:
embedded_problem, valid_flag = embed_problem_onto_hardware(problem_graph,
                                                            hardware_graph,
                                                            maxtime_sec=40,
                                                            attempts=10)

In [8]:
print(f'Was successful: {bool(valid_flag)}')

Was successful: True


In [9]:
dwave.embedding.verify_embedding(embedded_problem,
                                 problem_graph,
                                 hardware_graph)

True

In [10]:
embedded_problem

{0: [1728, 3578],
 1: [3728],
 2: [1653],
 3: [1743],
 4: [3624],
 5: [3684],
 6: [3699],
 7: [1698],
 8: [3654],
 9: [1608],
 10: [3577],
 11: [3579],
 12: [1623],
 13: [3609],
 14: [3669],
 15: [1727],
 16: [1713],
 17: [3593],
 18: [1758],
 19: [3743],
 20: [1683]}

In [11]:
embedded_qubo, convert_fn_sample = define_embedded_qubo_problem(
                                    embedded_problem,
                                    hardware_graph_edge_dict,
                                    dwave_qubo)

In [12]:
embedded_bqm = dimod.BinaryQuadraticModel.from_qubo(embedded_qubo)
print(f'number of variables: {len(embedded_bqm.variables)}')

number of variables: 22


# run solver on embedded problem

### Optional reverse anneal params

see further info at:
- [link](https://github.com/dwave-examples/reverse-annealing-notebook) for further details
- [link2](https://docs.dwavesys.com/docs/latest/c_qpu_annealing.html)


In [23]:
import numpy as np

reverse_anneal = True

if reverse_anneal is True:

    ######## kwargs 
    ## anneal schedule can be changed here!
    reverse_schedule = [[0.0, 1.0], [3, 0.45], [8, 0.45], [11, 1.0]]

     ### random initial state (note for real problems should use good classical input / known structure of problem)
    starting_state_non_embedded = {var: np.random.randint(0,2) for var in source_bqm.variables}

    ## whether to re-init state in reverse anneal
    reinitialize_state = True

    
    reverse_anneal_params = build_reverse_annealing_schedule_embedding(
        starting_state_non_embedded,
        source_bqm,
        reverse_schedule,
        reinitialize_state,
        
        embedded_bqm,
        hardware_graph,
        embedded_problem
    )
else:
    reverse_anneal_params = dict()

In [ ]:
## check aleady done in build_reverse_annealing_schedule_embedding funciton...
##  BUT doing again in notebook to see it working explicitly

# _, unembedded_test = unembed_samples(reverse_anneal_params['initial_state'],
#                                                      embedded_problem, 
#                                                      source_bqm)
# unembedded_test == starting_state_non_embedded

In [ ]:
### chain strength
# https://arxiv.org/pdf/2007.01730
# https://dwave-systemdocs.readthedocs.io/en/master/reference/embedding.html

from dwave.embedding.chain_strength import uniform_torque_compensation, scaled

chain_strength = scaled(source_bqm, embedding=embedded_bqm, prefactor=1.414)
chain_strength = uniform_torque_compensation(source_bqm, embedding=embedded_bqm, prefactor=1.414)
chain_strength

In [ ]:
DWAVE_API_TOKEN = input('please enter DWAVE TOKEN:')
optimizer = dimod_optimizer_cloud(DWAVE_API_TOKEN,
                                  reverse_anneal_params=reverse_anneal_params)

In [ ]:
n_samples = 100

embedded_sampleset, embedded_best_sample = optimizer.sample_qubo(embedded_qubo,
                                                                  n_samples,
                                                                  chain_strength=chain_strength)

# Convert output into original problem form

In [ ]:
unembedded_samples, unembedded_best = unembed_samples(embedded_sampleset,
                                                     embedded_problem, 
                                                   source_bqm)
unembedded_samples

In [ ]:
source_bqm.energy(unembedded_best)

# check against exact answer:

In [ ]:
solver = dimod.ExactSolver()
out = dimod.ExactSolver().sample_qubo(dwave_qubo)
source_bqm.energy(out.first.sample)

In [ ]:
embedded_bqm.energy(embedded_best_sample)

In [ ]:
# dimod.BinaryQuadraticModel.from_serializable(source_bqm.to_serializable())

# save output

Extra data added to seperate dict:
- the parent BQM
- the embedded problem
- problem_graph
- hardware_graph

In [ ]:
# nx.to_dict_of_dicts(problem_graph)

In [ ]:
extra_dictionary = {
    'source_bqm'      : source_bqm.to_serializable(),
    'embedded_bqm'    : embedded_bqm.to_serializable(),
    # 'embedded_qubo'   : embedded_qubo,
    # 'dwave_qubo'      : dwave_qubo,
    
    'embedded_problem': embedded_problem,
    'problem_graph'   : nx.adjacency_matrix(problem_graph, weight='bias').todense().tolist(),
    'hardware_graph'  : nx.adjacency_matrix(hardware_graph, weight='bias').todense().tolist(),
    
        }

In [ ]:
output_file_path = save_annealing_results(embedded_sampleset,
                        extra_ending='test_exp',
                        verbose=False,
                        extra_dictionary=extra_dictionary,
                        zip_data=True)

In [ ]:
from dwave.embedding import EmbeddedStructure
from dwave.embedding.utils import adjacency_to_edges
target_edges = adjacency_to_edges(hardware_graph)
EmbeddedStructure_problem = EmbeddedStructure(target_edges, embedded_problem)
EmbeddedStructure_problem

In [ ]:
EmbeddedStructure_problem = EmbeddedStructure(hardware_graph.edges(), ## <--- same as above!
                                               embedded_problem)
EmbeddedStructure_problem

In [ ]:
EmbeddedStructure_problem = EmbeddedStructure(hardware_graph.edges(), ## <--- same as above!
                                               embedded_problem)
EmbeddedStructure_problem

In [ ]:
initial_state = {var: np.random.randint(0,2) for var in source_bqm.variables}

### get energy of this state!
E0 = source_bqm.energy(initial_state)
## Put everything in the correct format
initial_state_sampleset = dimod.SampleSet.from_samples(initial_state, 
                                                    energy=[E0], 
                                                    vartype=dimod.vartypes.Vartype.BINARY)


initial_state_embedded = {val: initial_state[key] for key, terms in EmbeddedStructure_problem.items() for val in terms}
E0_embedded = embedded_bqm.energy(initial_state_embedded)

embedded_inital_state_sampleset = dimod.SampleSet.from_samples(initial_state_embedded, 
                                                    energy=[E0_embedded], 
                                                    vartype=dimod.vartypes.Vartype.BINARY)

In [ ]:
checker, best = unembed_samples(embedded_inital_state_sampleset,
                            embedded_problem, 
                            source_bqm)
checker

In [ ]:
best == initial_state_sampleset.first.sample